### Importing Dependencies

In [1]:
!pip install torch
!pip install torchdata

In [2]:
!pip install transformers==4.27.2
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

Summarize Dialogue with llm flan t5 base model from hugging face ,use DialogueSum datasets from higgin face


In [4]:
hugging_face_dataset_name="knkarthick/dialogsum"
dataset=load_dataset(hugging_face_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Print some dialogue with their baseline summaries

In [5]:
example_indices = [40 , 100]
dash_line = '_'.join('' for x in range(100))

for i, index in enumerate(example_indices):
  print(dash_line)
  print('Example', i+1)
  print('INPUT DIALOGUE:')
  print(dataset['test'][index]['dialogue'])
  print(dash_line)
  print("BASELINE HUMAN SUMMARY:")
  print(dataset['test'][index]['summary'])
  print(dash_line)
  print()

___________________________________________________________________________________________________
Example 1
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________

___________________________________________________________________________________________________
Example 2
INPUT DIALOGUE:
#Person1#: OK, that's a cut! Let's start from the beginning, everyone.
#Person

In [7]:
model_name='google/flan-t5-base'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)

## Load FLAN T5 mode , create instance for AutoModelSeq2SeqLM class with .from_pretrained() model

In [8]:
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [11]:
sentence='What time is it Tom?'
sentence_encoded=tokenizer(sentence, return_tensors='pt')
sentence_decoded=tokenizer.decode(
    sentence_encoded["input_ids"][0],
    skip_special_token=True
)
print('ENCODED SENTENCE')
print(sentence_encoded["input_ids"][0])
print('\n DECODED SENTENCE')
print(sentence_decoded)

ENCODED SENTENCE
tensor([ 363,   97,   19,   34, 3059,   58,    1])

 DECODED SENTENCE
What time is it Tom?</s>


In [15]:
for i, index in enumerate(example_indices):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']
  inputs=tokenizer(dialogue, return_tensors='pt')
  output=tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=50,
        )[0],
    skip_special_token=True
  )

  print(dash_line)
  print('Example', i+1)
  print(dash_line)
  print(f'INPUT Prompt \n{dialogue}')
  print(dash_line)
  print(f"BASELINE HUMAN SUMMARY:\n{summary}")
  print(dash_line)
  print(f"Generate witout prompting\n{output}\n")

___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
INPUT Prompt 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
Generate witout prompting
<pad> Person1: It's ten to nine.</s>

___________________________________________

In [16]:
#SUMMARIZE DIALOGUE WITH INSTRUCTION PROMPT

## **Zero Shot Inference**

In [17]:
for i, index in enumerate(example_indices):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']
  prompt=f"""summarize the following conversation.
{dialogue}
  summary:"""
  inputs=tokenizer(dialogue, return_tensors='pt')
  output=tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=50,
        )[0],
    skip_special_token=True
  )

  print(dash_line)
  print('Example', i+1)
  print(dash_line)
  print(f'INPUT Prompt \n{dialogue}')
  print(dash_line)
  print(f"BASELINE HUMAN SUMMARY:\n{summary}")
  print(dash_line)
  print(f"Model Generation ---Zero Shot: \n{output}\n")

___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
INPUT Prompt 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
Model Generation ---Zero Shot: 
<pad> Person1: It's ten to nine.</s>

_____________________________________

In [18]:
for i, index in enumerate(example_indices):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']
  prompt=f"""
{dialogue}
  What was going on"""
  inputs=tokenizer(dialogue, return_tensors='pt')
  output=tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=50,
        )[0],
    skip_special_token=True
  )

  print(dash_line)
  print('Example', i+1)
  print(dash_line)
  print(f'INPUT Prompt \n{dialogue}')
  print(dash_line)
  print(f"BASELINE HUMAN SUMMARY:\n{summary}")
  print(dash_line)
  print(f"Model Generation ---Zero Shot: \n{output}\n")

___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
INPUT Prompt 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
Model Generation ---Zero Shot: 
<pad> Person1: It's ten to nine.</s>

_____________________________________

# ONE SHOT INFERENCE

In [22]:
def make_prompt(example_indices_full,example_index_to_summarize):

  prompt=''

  for index in example_indices_full:
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']

    prompt=f"""
  Dialogue:

{dialogue}

  What was going on
  {summary}

  """
  dialogue=dataset['test'][example_index_to_summarize]['dialogue']
  prompt += f"""
  Dialogue:

{dialogue}

  What was going on
  """
  return prompt



In [23]:
example_indices_full=[40]
example_index_to_summarize=200
one_inference_shot=make_prompt(example_indices_full,example_index_to_summarize)
print(one_inference_shot)


  Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

  What was going on
  #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

  
  Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you

now pass this prompt to make one short inference

In [28]:
summary=dataset['test'][example_index_to_summarize]['summary']
inputs=tokenizer(one_inference_shot, return_tensors='pt')
output=tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=50,
        )[0],
    skip_special_token=True
  )

print(dash_line)
print(f"BASELINE HUMAN SUMMARY:\n{summary}")
print(dash_line)
print(f"Model Generation ---Zero Shot: \n{output}\n")

___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________
Model Generation ---Zero Shot: 
<pad> #Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.</s>



# Few Shot Inference

In [29]:
example_indices_full=[40,80,120]
example_index_to_summarize=200
few_inference_shot=make_prompt(example_indices_full,example_index_to_summarize)
print(few_inference_shot)


  Dialogue:

#Person1#: Hello, I bought the pendant in your shop, just before. 
#Person2#: Yes. Thank you very much. 
#Person1#: Now I come back to the hotel and try to show it to my friend, the pendant is broken, I'm afraid. 
#Person2#: Oh, is it? 
#Person1#: Would you change it to a new one? 
#Person2#: Yes, certainly. You have the receipt? 
#Person1#: Yes, I do. 
#Person2#: Then would you kindly come to our shop with the receipt by 10 o'clock? We will replace it. 
#Person1#: Thank you so much. 

  What was going on
  #Person1# wants to change the broken pendant in #Person2#'s shop.

  
  Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdate

In [31]:
summary=dataset['test'][example_index_to_summarize]['summary']
inputs=tokenizer(few_inference_shot, return_tensors='pt')
output=tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=50,
        )[0],
    skip_special_token=True
  )

print(dash_line)
print(f"BASELINE HUMAN SUMMARY:\n{summary}")
print(dash_line)
print(f"Model Generation ---Few Shot: \n{output}\n")

___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________
Model Generation ---Few Shot: 
<pad> #Person1 wants to upgrade his computer. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.</s>



Generation Configuration

In [40]:
generation_config=GenerationConfig(max_new_token=50)
#generation_config=GenerationConfig(max_new_token=10)
#generation_config=GenerationConfig(max_new_token=50, do_sample=True, temperature=0.1)
#generation_config=GenerationConfig(max_new_token=80, do_sample=True, temperature=0.8)
#generation_config=GenerationConfig(max_new_token=50, do_sample=True, temperature=0.5)
#generation_config=GenerationConfig(max_new_token=50, do_sample=True, temperature=1.0)





summary=dataset['test'][example_index_to_summarize]['summary']
inputs=tokenizer(few_inference_shot, return_tensors='pt')
output=tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        generation_config=generation_config,
        )[0],
    skip_special_token=True
  )

print(dash_line)
print(f"Model Generation ---Few Shot: \n{output}\n")
print(dash_line)
print(f"BASELINE HUMAN SUMMARY:\n{summary}")



___________________________________________________________________________________________________
Model Generation ---Few Shot: 
<pad> #Person1 wants to upgrade his computer. #Person2 wants to add a

___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
